In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url='http://api.scraperapi.com', #scrapeapi
            params={
                'api_key': 'fa9b49e2d89dc81fd7808deefca5c790',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v2.csv')
            line_count+=1

    

In [8]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("tmp_11_v6.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v6.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('tmp_12_v6.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [9]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/ballito-central/ballito/kwazulu-natal/17817/109369954
{'name': '6 Bedroom House for Sale in Ballito Central', 'price': 'R 28 000 000', 'province': 'kwazulu-natal', 'city': 'ballito', 'address': 'Ballito Central, Ballito', 'Listing Number': '109369954', 'Type of Property': 'House', 'Lifestyle': 'Country Living', 'Listing Date': '19 November 2020', 'Erf Size': '6 840 m²', 'Floor Size': '650 m²', 'Rates and Taxes': 'R 3 200', 'Age': '17', 'Pets Allowed': 'Yes', 'Coverage': '9 %', 'Bedrooms': '6', 'Bedroom 1': 'Tiled Floors, Air Conditioner, Fan, King Bed', 'Bedroom 2': 'Tiled Floors, Air Conditioner, Fan, King Bed', 'Bedroom 3': 'Cottage', 'Bedroom 4': 'Tiled Floors, Air Conditioner, Fan, King Bed', 'Bedroom 5': 'Tiled Floors, Air Conditioner, Fan, King Bed', 'Bedroom 6': 'Tiled Floors, Air Conditioner, Fan, King Bed', 'Bathrooms': '6', 'Bathroom 1': 'Full, Main en Suite', 'Bathroom 2': 'En suite, Shower, Toilet and Basin', 'Bathroom 3': 'En suite, Shower, Toi

{'name': '4 Bedroom House for Sale in Greenways Golf Estate', 'price': 'R 6 900 000', 'province': 'western-cape', 'city': 'strand', 'address': '\n4 Bedroom House for Sale in Greenways Golf Estate\n', 'Listing Number': '108847431', 'Type of Property': 'House', 'Street Address': ' 8 Muirfield Crescent, Greenways Golf Estate', 'Description': 'Double Storey, Freestanding', 'Lifestyle': 'Coastal/Beach, Golf Estate, Security Estate', 'Listing Date': '08 July 2020', 'Erf Size': '639 m²', 'Floor Size': '688 m²', 'Levies': 'R 1 450', 'Rates and Taxes': 'R 3 700', 'Pets Allowed': 'Yes', 'Bedrooms': '4', 'Bedroom 1': 'Master Main Bedroom', 'Bedroom 2': 'Wooden Floors, Air Conditioner, Built-in Cupboards', 'Bedroom 3': 'Wooden Floors, Air Conditioner, Built-in Cupboards', 'Bedroom 4': 'Guest suite with lounge & kitchen', 'Bathrooms': '3.5', 'Bathroom 1': 'Full, Main en Suite', 'Bathroom 2': 'Full', 'Bathroom 3': 'Full, En suite', 'Bathroom 4': 'Guest Toilet', 'Kitchens': '2', 'Kitchen 1': 'Kitchen

{'name': '4 Bedroom House for Sale in Mackenzie Park', 'price': 'R 2 595 000', 'province': 'gauteng', 'city': 'benoni', 'address': 'Mackenzie Park, Benoni', 'Listing Number': '109809358', 'Type of Property': 'House', 'Description': 'Double Storey', 'Lifestyle': 'Suburban', 'Listing Date': '31 March 2021', 'Erf Size': '788 m²', 'Levies': 'R 250', 'Rates and Taxes': 'R 1 198', 'Bedrooms': '4', 'Bedroom 1': 'Main Bedroom upstairs leads off to balcony', 'Bedroom 2': 'Bedroom upstairs leads off to balcony', 'Bedroom 3': 'Bedroom downstairs', 'Bedroom 4': 'Curtain Rails, Built-in Cupboards, Queen Bed, Laminated Floors', 'Bathrooms': '4', 'Bathroom 1': 'Guest Toilet', 'Bathroom 2': 'En suite to Main Bedroom upstairs. Bathroom has skylight', 'Bathroom 3': 'En suite to the one bedroom downstairs', 'Bathroom 4': 'Tiled Floors, Basin, Bath, Shower, Toilet', 'Kitchen': 'Breakfast Nook, Scullery, Eye Level Oven, Extractor Fan, Dishwasher Connection, Granite Tops, Washing Machine Connection, Tiled F

{'name': '0.5 Bedroom Apartment / Flat for Sale in Fresnaye', 'price': 'R 2 100 000', 'province': 'western-cape', 'city': 'cape-town', 'address': 'Fresnaye, Cape Town', 'Listing Number': '109988820', 'Type of Property': 'Apartment / Flat', 'Listing Date': '18 May 2021', 'Floor Size': '55 m²', 'Price per m²': 'R 38 182', 'Levies': 'R 1 000', 'Rates and Taxes': 'R 1 200', 'Pets Allowed': 'Yes', 'Furnished': 'Yes', 'Bedroom': '0.5', 'Bathrooms': '1', 'Kitchens': '1', 'Lounges': '1', 'Dining Rooms': '1', 'Education': ['0.49km', '1.20km', '1.68km', '1.73km', '1.79km', '2.09km', '2.10km', '2.12km'], 'Transport and Public Services': ['0.75km', '0.77km', '0.86km', '1.57km', '1.68km', '2.09km'], 'Food and Entertainment': ['1.28km', '1.30km', '1.33km', '1.34km', '1.68km', '1.88km', '1.96km', '1.99km', '2.05km', '2.06km', '2.09km', '2.10km', '2.10km', '2.11km', '2.11km', '2.13km']}
==>Property link:  /for-sale/pezula-private-estate/knysna/western-cape/11144/109976665
{'name': '3 Bedroom House for

{'name': '4 Bedroom House for Sale in Raisethorpe', 'price': 'R 1 100 000', 'province': 'kwazulu-natal', 'city': 'pietermaritzburg', 'address': 'Raisethorpe, Pietermaritzburg', 'Listing Number': '110096224', 'Type of Property': 'House', 'Listing Date': '15 June 2021', 'Erf Size': '601 m²', 'Pets Allowed': 'No', 'Bedrooms': '4', 'Bathrooms': '2', 'Kitchens': '1', 'Lounges': '1', 'Dining Rooms': '1', 'Parking': '3', 'Garden': 'Yes', 'Education': ['0.39km', '0.42km', '0.59km', '0.62km', '0.64km', '0.67km', '0.92km', '0.96km', '1.09km', '1.16km', '1.30km', '1.50km', '1.56km', '1.62km', '1.71km', '2.06km', '2.09km', '2.10km', '2.13km', '2.13km', '2.22km', '2.30km', '2.31km', '2.34km', '2.43km', '2.54km', '2.57km', '2.68km', '2.73km', '2.74km']}
==>Property link:  /for-sale/signal-hill/plettenberg-bay/western-cape/10551/990598426
{'name': '6 Bedroom House for Sale in Signal Hill', 'price': 'R 4 750 000', 'province': 'western-cape', 'city': 'plettenberg-bay', 'address': '\n6 Bedroom House for

AttributeError: 'NoneType' object has no attribute 'find'